In [46]:
import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt
from configparser import ConfigParser
import requests
from bs4 import BeautifulSoup
import http.cookiejar
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,auc

In [33]:
#Get input from configuration
url = 'https://freddiemac.embs.com/FLoan/Data/download.php'

cfg = ConfigParser()
cfg.read('classification.config')
value = cfg.get('cookie','value')
docu = cfg.get('file','name')

In [28]:
if len(docu)!=6 :
    raise Exception("File name error")
    exit()


In [29]:
docu2=''
if docu[1]=='4':
    docu2 = docu[0]+'1'+str(int(docu[2:])+1)
else:
    docu2 = docu[0]+str(int(docu[1])+1)+docu[2:]
dww=[docu,docu2]        

In [34]:
cookie={'PHPSESSID': value}

r = requests.post(url,cookies=cookie)
content=r.content
soup = BeautifulSoup(content,'lxml')
all_href = soup.find_all('a')
i=0;
lod=len(dww)
url_list=['https://freddiemac.embs.com/FLoan/Data/']*lod

for href in all_href:
    for s in dww:
        if s in href['href']:
            url_list[i]=url_list[i]+href['href']
            i+=1    


In [35]:
if len(all_href)==0:
     raise Exception("Login error")   

In [36]:
if (not os.path.exists('data')):
    os.mkdir('data')
os.chdir('data')
for i in range(len(url_list)):
    r = requests.get(url_list[i],cookies=cookie)
    with open(url_list[i][71:77]+'.zip','wb') as code:
        code.write(r.content)
files= os.listdir()
for file in files:
    if '.zip' in file:
        shutil.unpack_archive(file)
os.chdir('..')

In [37]:
file1="data/historical_data1_time_"+docu+".txt"
file2="data/historical_data1_time_"+docu2+".txt"

In [38]:
data = pd.read_csv(file1,sep="|",header=None)
data.columns = ["LOAN SEQUENCE NUMBER","MONTHLY REPORTING PERIOD","CURRENT ACTUAL UPB","CURRENT LOAN DELINQUENCY STATUS"
                ,"LOAN AGE",
               "REMAINING MONTHS TO LEGAL MATURITY","REPURCHASE FLAG","MODIFICATION FLAG"
                ,"ZERO BALANCE CODE","ZERO BALANCE EFFECTIVE DATE",
               "CURRENT INTEREST RATE","CURRENT DEFERRED UPB","DUE DATE OF LAST PAID INSTALLMENT (DDLPI)"
                ,"MI RECOVERIES","NET SALES PROCEEDS",
               "NON MI RECOVERIES","EXPENSES","LEGAL COSTS","MAINTENANCE AND PRESERVATION COSTS"
                ,"TAXES AND INSURANCE","MISCELLANEOUS EXPENSES",
               "ACTUAL LOSS CALCULATION","MODIFICATION COST","STEP MODIFICATION FLAG"
                ,"DEFERRED PAYMENT MODIFICATION","ESTIMATED LOAN TO VALUE (ELTV)"]

C:\Users\wenqi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,7,14,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [40]:
vali = pd.read_csv(file2,sep="|",header=None)
vali.columns = ["LOAN SEQUENCE NUMBER","MONTHLY REPORTING PERIOD","CURRENT ACTUAL UPB","CURRENT LOAN DELINQUENCY STATUS"
                ,"LOAN AGE",
               "REMAINING MONTHS TO LEGAL MATURITY","REPURCHASE FLAG","MODIFICATION FLAG"
                ,"ZERO BALANCE CODE","ZERO BALANCE EFFECTIVE DATE",
               "CURRENT INTEREST RATE","CURRENT DEFERRED UPB","DUE DATE OF LAST PAID INSTALLMENT (DDLPI)"
                ,"MI RECOVERIES","NET SALES PROCEEDS",
               "NON MI RECOVERIES","EXPENSES","LEGAL COSTS","MAINTENANCE AND PRESERVATION COSTS"
                ,"TAXES AND INSURANCE","MISCELLANEOUS EXPENSES",
               "ACTUAL LOSS CALCULATION","MODIFICATION COST","STEP MODIFICATION FLAG"
                ,"DEFERRED PAYMENT MODIFICATION","ESTIMATED LOAN TO VALUE (ELTV)"]

C:\Users\wenqi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,7,14,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
def preprocessing(data):
    data['Delinquent']=[0 if x == '0' else 1 for x in data['CURRENT LOAN DELINQUENCY STATUS']]
    data.drop(['LOAN SEQUENCE NUMBER','REPURCHASE FLAG','CURRENT LOAN DELINQUENCY STATUS','MODIFICATION FLAG','ZERO BALANCE CODE'
           ,'ZERO BALANCE EFFECTIVE DATE','DUE DATE OF LAST PAID INSTALLMENT (DDLPI)','MI RECOVERIES'
          ,'NET SALES PROCEEDS','NON MI RECOVERIES','EXPENSES','LEGAL COSTS','MAINTENANCE AND PRESERVATION COSTS'
          ,'TAXES AND INSURANCE','MISCELLANEOUS EXPENSES','ACTUAL LOSS CALCULATION','MODIFICATION COST'
          ,'STEP MODIFICATION FLAG','ESTIMATED LOAN TO VALUE (ELTV)'],axis=1,inplace=True)
    dictionary_25 = {'Y':1,'N':0,' ':-1}
    data['DEFERRED PAYMENT MODIFICATION'] = [dictionary_25[x] for x in data['DEFERRED PAYMENT MODIFICATION']]
    return data
    

In [42]:
train_data=data[:300000]
train_data = preprocessing(train_data)
vali_data=vali[:300000]
vali_data = preprocessing(vali_data)

C:\Users\wenqi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\wenqi\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\wenqi\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '

In [43]:
train_x = train_data.drop('Delinquent',axis=1)
train_y = train_data['Delinquent']
test_x = vali_data.drop('Delinquent',axis=1)
test_y = vali_data['Delinquent']

In [47]:
rf = RandomForestClassifier(max_depth=7,n_estimators=100)
rf.fit(train_x,train_y)
pred_y = rf.predict(test_x)
cm=confusion_matrix(test_y,pred_y)
tn, fp, fn, tp = cm.ravel()
print('TN: ',tn)
print('FP: ',fp)
print('FN: ',fn)
print('TP: ',tp)

TN:  285600
FP:  526
FN:  13325
TP:  549


In [52]:
print('Number of Actual Delinquents:', tp+fn)
print('Number of Predicted Delinquents:',  tp+fp)
print('Number of records in the dataset:',  tn+tp+fn+fp)
print('Number of Delinquents properly classified:',  tp)
print('Number of nondelinquents improperly classified as delinquents:',  fp)

Number of Actual Delinquents: 13874
Number of Predicted Delinquents: 1075
Number of records in the dataset: 300000
Number of Delinquents properly classified: 549
Number of nondelinquents improperly classified as delinquents: 526
